# SQL Event-based data:

## Objetivos:
    1. Utilizar un pandas data-frame como base de datos.
    2. Enviar un query utilizando pandas.
    3. Event-based data: propiedades, características y retos.
    4. Consideraciones: Subqueries & partitions.

## 1. Librerias y configuraciones

In [3]:
%conda install pandasql

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.13.0
  latest version: 4.14.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from pandasql import sqldf

In [5]:
mysql = lambda q: sqldf(q, globals())

## 2. Entendiendo la data de eventos

In [9]:
event_df = pd.read_csv("events.csv")

In [10]:
event_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [5]:
event_df[event_df["transactionid"].notna()].head()

,timestamp,visitorid,event,itemid,transactionid
130,1433222276276,599528,transaction,356475,4000.0
304,1433193500981,121688,transaction,15335,11117.0
418,1433193915008,552148,transaction,81345,5444.0
814,1433176736375,102019,transaction,150318,13556.0
843,1433174518180,189384,transaction,310791,7244.0


## 3. Trabajando con SQLDF

In [6]:
mysql("""
    SELECT COUNT(*) FROM event_df;
""")

,COUNT(*)
0,2756101


In [7]:
mysql("""
    SELECT
        event
        ,COUNT(DISTINCT visitorid) visits
        ,COUNT(DISTINCT itemid) items
        ,COUNT(DISTINCT transactionid) txns
    FROM event_df
    GROUP BY 1
    ORDER BY 2 DESC
    ;
""")

,event,visits,items,txns
0,view,1404179,234838,0
1,addtocart,37722,23903,0
2,transaction,11719,12025,17672


In [13]:
mysql("""
    SELECT
        visitorid
        ,MAX(CASE WHEN event = 'view' THEN 1 ELSE 0 END) views
        ,MAX(CASE WHEN event = 'addtocart' THEN 1 ELSE 0 END) cart
        ,MAX(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) txn
    FROM event_df
    GROUP BY 1
    LIMIT 10
    ;
""")

,visitorid,views,cart,txn
0,0,1,0,0
1,1,1,0,0
2,2,1,0,0
3,3,1,0,0
4,4,1,0,0
5,5,1,0,0
6,6,1,1,0
7,7,1,0,0
8,8,1,0,0
9,9,1,0,0


In [14]:
mysql("""
    SELECT
        COUNT(DISTINCT CASE WHEN views = 1 THEN visitorid END) visits
        ,COUNT(DISTINCT CASE WHEN views = 1 AND cart = 1 THEN visitorid END) cart
        ,COUNT(DISTINCT CASE WHEN views = 1 AND cart = 1 AND txn = 1 THEN visitorid END) txns
    FROM (
        SELECT
            visitorid
            ,MAX(CASE WHEN event = 'view' THEN 1 ELSE 0 END) views
            ,MAX(CASE WHEN event = 'addtocart' THEN 1 ELSE 0 END) cart
            ,MAX(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) txn
        FROM event_df
        GROUP BY 1
    ) a
    ;
""")

,visits,cart,txns
0,1404179,34401,10228


In [16]:
mysql("""
    SELECT 
        COUNT(DISTINCT visitorid) vistors_with_txns
    FROM event_df
    WHERE 
        event = 'transaction'
    ;
""")

,vistors_with_txns
0,11719


In [17]:
mysql("""
    SELECT 
        views
        ,cart
        ,COUNT(DISTINCT visitorid) visits
    FROM (
        SELECT
            visitorid
            ,MAX(CASE WHEN event = 'view' THEN 1 ELSE 0 END) views
            ,MAX(CASE WHEN event = 'addtocart' THEN 1 ELSE 0 END) cart
            ,MAX(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) txn
        FROM event_df
        GROUP BY 1
    ) a
    WHERE
        txn = 1
    GROUP BY 1,2
    ;
""")

,views,cart,visits
0,0,0,80
1,0,1,348
2,1,0,1063
3,1,1,10228


In [11]:
mysql("""
    SELECT
        visitorid
        ,event
        ,timestamp
        ,itemid
        ,transactionid
        ,ROW_NUMBER() OVER(PARTITION BY visitorid, event ORDER BY timestamp) seq
    FROM event_df
    GROUP BY 1,2,3,4,5
    LIMIT 50
    ;
""")

,visitorid,event,timestamp,itemid,transactionid,seq
0,0,view,1442004589439,285930,None,1
1,0,view,1442004759591,357564,None,2
2,0,view,1442004917175,67045,None,3
3,1,view,1439487966444,72028,None,1
4,2,view,1438969904567,325215,None,1
5,2,view,1438970013790,325215,None,2
6,2,view,1438970212664,259884,None,3
7,2,view,1438970468920,216305,None,4
8,2,view,1438970905669,342816,None,5
9,2,view,1438971444375,342816,None,6
